In [32]:
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  LabelEncoder, OneHotEncoder
from sklearn.metrics import  confusion_matrix, accuracy_score,classification_report

import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import pandas_profiling

import warnings
warnings.filterwarnings('ignore')

In [5]:
data = pd.read_csv('Churn_Modelling.data')

In [6]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
data.shape

(10000, 14)

In [11]:
pd.read_csv('Churn_Modelling.data').profile_report()

In [12]:
data.drop(['Surname', 'CustomerId', 'RowNumber'], axis = 1, inplace = True)

In [13]:
data.shape

(10000, 11)

# Data Preprocessing

In [14]:
X = data.iloc[:, 0:-1].values
Y = data.iloc[:, -1:].values

In [16]:
X.shape, Y.shape

((10000, 10), (10000, 1))

In [17]:
X[0]

array([619, 'France', 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88],
      dtype=object)

In [18]:
Y[0]

array([1])

# Applying One hot encoding

In [20]:
lb = LabelEncoder()
X[:, 1] = lb.fit_transform(X[:, 1])
X[:, 2] = lb.fit_transform(X[:, 2])

In [21]:
X[0]

array([619, 0, 0, 42, 2, 0.0, 1, 1, 1, 101348.88], dtype=object)

In [22]:
ohe = OneHotEncoder(categorical_features = [1])
X = ohe.fit_transform(X).toarray()

In [24]:
X[0], X.shape

(array([1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 6.1900000e+02,
        0.0000000e+00, 4.2000000e+01, 2.0000000e+00, 0.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0134888e+05]),
 (10000, 12))

In [25]:
X = X[:,1:]

In [29]:
print(X[0])
print('Shape of X-', X.shape)

[0.0000000e+00 0.0000000e+00 6.1900000e+02 0.0000000e+00 4.2000000e+01
 2.0000000e+00 0.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
 1.0134888e+05]
Shape of X- (10000, 11)


# Splitting data in to training and testing

In [30]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [31]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((8000, 11), (2000, 11), (8000, 1), (2000, 1))

# Fitting XGboost Model to the training set

In [34]:
classifier = XGBClassifier()
classifier.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [35]:
y_pred = classifier.predict(x_test)

In [36]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [37]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.95      0.92      1595
           1       0.74      0.51      0.61       405

    accuracy                           0.86      2000
   macro avg       0.81      0.73      0.76      2000
weighted avg       0.86      0.86      0.85      2000



In [38]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[1521,   74],
       [ 197,  208]])